In [ ]:
# Databricks notebook source
# MAGIC %md
# MAGIC # Customer 360 Data Processing
# MAGIC This notebook processes customer data from various sources to create a comprehensive customer profile.

# COMMAND ----------

import logging
from pyspark.sql.functions import col, count, max, avg, datediff, current_date, when, lit

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# COMMAND ----------

try:
    # Step 1: Data Source Configuration
    logger.info("Loading data sources from Unity Catalog tables.")
    policy_df = spark.table("catalog.db.policy")
    claims_df = spark.table("catalog.db.claims")
    demographics_df = spark.table("catalog.db.demographics")
    scores_df = spark.table("catalog.db.scores")
    aiml_insights_df = spark.table("catalog.db.aiml_insights")

# COMMAND ----------

    # Step 2: Data Selection and Filtering
    logger.info("Selecting relevant fields from demographics data.")
    selected_demographics_df = demographics_df.select(
        "Customer_ID", "Customer_Name", "Email", "Phone_Number", "Address", "City", "State", "Postal_Code",
        "Date_of_Birth", "Gender", "Marital_Status", "Occupation", "Income_Level", "Customer_Segment"
    )

# COMMAND ----------

    # Step 3: Data Integration
    logger.info("Joining demographics and policy data.")
    joined_df = selected_demographics_df.join(
        policy_df, selected_demographics_df.Customer_ID == policy_df.customer_id, "inner"
    ).drop(policy_df.customer_id)

    logger.info("Joining the result with claims data.")
    joined_df = joined_df.join(
        claims_df, joined_df.policy_id == claims_df.Policy_ID, "inner"
    ).drop(claims_df.Policy_ID)

# COMMAND ----------

    # Step 4: Data Aggregation
    logger.info("Aggregating data to compute metrics.")
    aggregated_df = joined_df.groupBy("Customer_ID").agg(
        count("Claim_ID").alias("Total_Claims"),
        count("policy_id").alias("Policy_Count"),
        max("Claim_Date").alias("Recent_Claim_Date"),
        avg("Claim_Amount").alias("Average_Claim_Amount")
    )

# COMMAND ----------

    # Step 5: Custom Calculations
    logger.info("Calculating custom metrics.")
    final_df = aggregated_df.withColumn("Age", datediff(current_date(), col("Date_of_Birth")) / 365) \
        .withColumn("Claim_To_Premium_Ratio", when(col("total_premium_paid") != 0, col("Average_Claim_Amount") / col("total_premium_paid")).otherwise(0)) \
        .withColumn("Claims_Per_Policy", when(col("Policy_Count") != 0, col("Total_Claims") / col("Policy_Count")).otherwise(0)) \
        .withColumn("Retention_Rate", lit(0.85)) \
        .withColumn("Cross_Sell_Opportunities", lit("Multi-Policy Discount, Home Coverage Add-on")) \
        .withColumn("Upsell_Potential", lit("Premium Vehicle Coverage"))

# COMMAND ----------

    # Step 6: Comprehensive Data Consolidation
    logger.info("Integrating all data sources into a comprehensive customer profile.")
    customer_360_df = final_df.join(scores_df, "Customer_ID", "inner") \
        .join(aiml_insights_df, "Customer_ID", "inner")

# COMMAND ----------

    # Step 7: Output Generation
    logger.info("Writing the final output to Delta format.")
    customer_360_df.write.format("delta").mode("overwrite").saveAsTable("catalog.db.customer_360")

except Exception as e:
    logger.error(f"An error occurred: {e}")
    raise
